# Notebook handeling

In [2]:
#| default_exp notebook

## Notebook object

The underlying file structure of a notebook is JSON, which directly matches python dictionaries.  
As such, it is treated by `nbformat` as spcieal kind of `dict`.  

In [3]:
#| export

from nbformat import NotebookNode
from nbformat import read as read_nb, write as write_nb, validate as validate_nb, from_dict
import jq

Due to that, all of `jq`s' power is available to us: 

In [4]:
#| export
from typing import Iterable
from enum import Enum

# TODO: about filter, how about:
# nb.cells = jq.compile(filter).input(nb.cells).all()?
class Notebook:
    def __init__(self, path):
        self.path = path
        self.nb = read_nb(path, as_version=4)

    def apply_jq_pattern(self, pattern):
        return from_dict(jq.compile(pattern).input(self.nb).first())

    def clean_execution_counts(self):
        self.nb = self.apply_jq_pattern('.cells[].execution_count = null')
        
    def clean_outputs(self):
        self.nb = self.apply_jq_pattern('.cells[].outputs = []')
    
    def clean_all_metadata(self):
        self.nb = self.apply_jq_pattern('.cells[].metadata = {}')
        self.nb = self.apply_jq_pattern('.metadata = {}')
        
    # TODO: circular dependency - notebooks.clean needs clean operations and vice versa
    def clean(self, clean_ops: Iterable[Enum]):        
        for op in clean_ops:
            getattr(self, op.value)()
        return self
        
    def save(self, path=None):
        path = path if path else self.path
        write_nb(self.nb, path)
        

In [5]:
#| export

class CleanOperations(Enum):
    METADATA = Notebook.clean_all_metadata.__name__
    OUTPUTS = Notebook.clean_outputs.__name__
    EXECUTION_COUNT = Notebook.clean_execution_counts.__name__    


## Tests

### Base Clean Notebook

In [45]:
#| hide

from pathlib import Path
test_data_dir = Path('../tests/data')

# tests the "clean" nb is indeed clean
cln_nb = read_nb(test_data_dir/'clean.ipynb', as_version=4)

#### Cleaning a clean notebook produces same result

In [46]:
#| hide

# TODO: enable clean to recieve __all__ clean operations - maybe make an __all__ flag
assert Notebook(test_data_dir/'clean.ipynb').clean([CleanOperations.METADATA, CleanOperations.OUTPUTS, CleanOperations.EXECUTION_COUNT]).nb == cln_nb

### Clean All Attributes

In [48]:
from unittest import TestCase
tc = TestCase()

cln = Notebook(test_data_dir/'dirty.ipynb').clean([CleanOperations.METADATA, CleanOperations.OUTPUTS, CleanOperations.EXECUTION_COUNT]).nb

tc.assertDictEqual(cln, cln_nb)

### Clean Outputs

In [55]:
no_out = Notebook(test_data_dir/'dirty.ipynb').clean([CleanOperations.OUTPUTS]).nb
tc.assertDictEqual(no_out, read_nb(test_data_dir/'no_outputs.ipynb', as_version=4))

### Clean Execution Counts

In [57]:
no_ex_cnt = Notebook(test_data_dir/'dirty.ipynb').clean([CleanOperations.EXECUTION_COUNT]).nb
tc.assertDictEqual(no_ex_cnt, read_nb(test_data_dir/'no_execution_counts.ipynb', as_version=4))

### Clean Metadata

## export

In [98]:
from nbdev.export import nb_export

nb_export('nb_definition.ipynb')